In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pylcp
from pylcp.atom import atom
import scipy.constants as cts
from numpy import random as rnd
from pylcp.common import progressBar
import pandas as pd
import magpylib as mg
import numpy.random 
import magpylib as magpy
from scipy.interpolate import RegularGridInterpolator
from scipy.stats import rv_continuous
import concurrent.futures as cf

# Setting up constants

In [ ]:
import numpy as np

# Constants
k_B = 1.380649e-23  # Boltzmann constant in J/K
T = 723.15  # Temperature in Kelvin
m = 173 * 1.66054e-27  # Mass of Yb-173 atom in kg

# Mean velocity
mean_velocity = np.sqrt((8 * k_B * T) / (np.pi * m))

# Standard deviation of velocity
std_dev_velocity = np.sqrt(k_B * T / m)

print("Mean Velocity:", mean_velocity, "m/s")
print("Standard Deviation of Velocity:", std_dev_velocity, "m/s")

In [ ]:
mass_lab = 173*cts.value('atomic mass constant') # YB-173 mass in kg
v_a = mean_velocity
sd_yb = std_dev_velocity


klab = 2*np.pi*25068.2222
# Lab wavevector (without 2pi) in cm^{-1}   # Lifetime of 6P_{3/2} state (in seconds)
gammalab = (1.93*1e8)
Blab = -67*1e-3#(v0*cts.hbar*(klab*100))/cts.value('Bohr magneton')
b_0L =  48*1e-3
# (v0*cts.hbar*(klab*100))/cts.value('Bohr magneton')/2
# T # About 15 G/cm is a typical gradient for Rb
L = 30
x0 = 1/klab  # cm
t0 = 1/gammalab  # s
#100*(mass_lab*(v0**2))/(0.5*cts.hbar*(klab*100)*gammalab) +5
print(Blab, L,b_0L)

In [ ]:
# Now, here are our `natural' length and time scales:
x0 = 1/klab  # distance in cm
t0 = 1/gammalab # time in s
vc = x0/t0*(1/100) # velcoity in cm/s for conversion back to s.i units
Fc = cts.hbar*klab*100*gammalab # force in newtond for conversion back to s.i units

mass = mass_lab*(x0*1e-2)**2/cts.hbar/t0

# And now our wavevector, decay rate, and magnetic field gradient in these units:
k = klab*x0
gamma = gammalab*t0
alpha = cts.value('Bohr magneton')*Blab*t0/cts.hbar
b_0 = cts.value('Bohr magneton')*b_0L*t0/cts.hbar 
l = L/x0 

print(x0, t0, mass, k, gamma, alpha,b_0)

# Magnetic field profile

In [6]:
def mag(R,B_0,alpha,l):
    if R[2] > l :
       return  np.array([0,0,0])
    else:
        return np.array([B_0 + alpha*np.sqrt(1 -R[2]/l),0,0])

In [ ]:
zeeman_prof = magpy.Collection()
j = 30
n = 8
a = 0# distance it circles aroundthe center 
c = 30 #angle of first bar
d =  24#angle of second bar
b = -190# distnace the two section of the array are apart from each other
k = -100 #the first and second magnent are apart
e = 40
l = -5
p= -20# distance the thrid and 4th magnet are apart from each other
m  = -42
mystyle = mstyle = dict(
    mode="color+arrow",
    color=dict(north="magenta", middle="white", south="turquoise"),
    arrow=dict(width=2, color="k")
)
angles = np.linspace(0, 360, n, endpoint=False)
for i in angles:
    cube1= magpy.magnet.Cuboid(
    dimension=(6,6,125),
    polarization=(-(1.080),0,0),
    position=(70+a+l,0,140+k))
    cube2 = magpy.magnet.Cuboid(dimension= (4,4,25),
    polarization=(-1.17,0,0),
    position=(60+a+l,0,50+e),style_magnetization = mystyle)
    cube3= magpy.magnet.Cuboid(
    dimension=(6,6,125),
    polarization=(1.08,0,0),
    position=(91.5+m,0,320+b))
 
#     cube5 = magpy.magnet.Cuboid(dimension= (4,4,25),
#     polarization=(-1.17,0,0),
#     position=(90+a+p+m,0,190))

#     cube6 = magpy.magnet.Cuboid(dimension= (4,4,25),
#     polarization=(-1.17,0,0),
#     position=(80+a+p+m,0,200),
# )
   


    cube1.rotate_from_angax(i, 'z')
    cube1.rotate_from_angax(c, 'y',)
    cube1.rotate_from_angax(i, 'z', anchor=0)
 
 
    cube2.rotate_from_angax(i, 'z',anchor = 0)
    cube2.rotate_from_angax(i, 'z')
    cube3.rotate_from_angax(i, 'z', anchor=0)
    cube3.rotate_from_angax(-d, 'y')
    cube3.rotate_from_angax(i, 'z')
  
    #cube4.rotate_from_angax(i, 'z', anchor=0)
    #cube4.rotate_from_angax(-d, 'y')
    #cube4.rotate_from_angax(i, 'z')
    

    zeeman_prof.add(cube1)
    zeeman_prof.add(cube2)
    zeeman_prof.add(cube3)
    #zeeman_prof.add(cube4)
    # zeeman_prof.add(cube5)
    # zeeman_prof.add(cube6)
    j = j+1


# tfmot = magpy.Collection()
# N = 4
# angles = np.linspace(0,360,N,endpoint=False)
# pol = [0,180,0,180]
# l = 35
# halbach = magpy.Collection()
# mystyle = mstyle = dict(
#     mode="color+arrow",
#     color=dict(north="magenta", middle="white", south="turquoise"),
#     arrow=dict(width=2, color="k"),
# )
# for a in range(len(angles)):
#     cube= magpy.magnet.Cuboid(
#     dimension=(8,6,80),
#     polarization=(1.17,0,0),
#     position=(30+l,0,230),
#     style_magnetization = mystyle)
#     print(a)
#     cube.rotate_from_angax(angles[a], 'z',anchor = 0)
#     cube.rotate_from_angax(pol[a], 'z')

#     tfmot.add(cube)

b_tot = magpy.Collection()
b_tot.add(zeeman_prof,override_parent=True)
b_tot.show()

In [8]:
l = 30/x0
y= np.linspace(0,2*30,1000)
r = []
for i in y:
    r.append(np.array([0,0,i]))
magg = []
for i in r:
    magg.append(mag(i,b_0L,Blab,30)[0]*1e3)

In [ ]:
z = np.linspace(0,2*L,1000)
b_y = []
b_z = []
b_x = []
for i in z:
    b_x.append(cts.value('Bohr magneton')*zeeman_prof.getB([0,0,i*10])[0])
    b_y.append(cts.value('Bohr magneton')*zeeman_prof.getB([0,0,i*10])[1])
    b_z.append(cts.value('Bohr magneton')*zeeman_prof.getB([0,0,i*10])[2])
    

fig,ax = plt.subplots(1,1,figsize=(10,5))    
plt.plot(z,np.array(b_x))
plt.plot(z,np.array(b_y))
plt.plot(z,np.array(b_z))


plt.xlabel('z (cm)')
plt.plot(z,np.array(magg), '--')

plt.ylabel('$B$ (mT)')
plt.legend(['$B_x$','$B_y$','$B_z$','Analytical'])  
plt.ylabel('$B_x$ (mT)')
plt.grid()
bf = lambda R: cts.value('Bohr magneton')*b_tot.getB(R*10*x0)*t0/cts.hbar


# Setting YB 173 system

In [10]:
# Define the atomic Hamiltonian:
H_g_D2, mu_q_g_D2 = pylcp.hamiltonians.hyperfine_coupled(
0, 5/2, 0,  	-0.2592,
    Ahfs = 0, Bhfs=0, Chfs=0,
    muB=1)# ground state 1s0
H_e_D2, mu_q_e_D2 = pylcp.hamiltonians.hyperfine_coupled(
1, 5/2, 1.035,	-0.2592,
    Ahfs=59.52*1e6,Bhfs = 601.87*1e6 , Chfs= 0,
    muB=1) #excited state 1p1

dijq_D2 = pylcp.hamiltonians.dqij_two_hyperfine_manifolds(0, 1, 5/2)

E_e_D2 = np.unique(np.diagonal(H_e_D2))
E_g_D2 = np.unique(np.diagonal(H_g_D2))
hamiltonian_D2 = pylcp.hamiltonian(mass = mass)
hamiltonian_D2.add_H_0_block('g', H_g_D2)
hamiltonian_D2.add_H_0_block('e', H_e_D2)
hamiltonian_D2.add_d_q_block('g', 'e', dijq_D2, gamma = gamma, k = k)
hamiltonian_D2.add_mu_q_block('g', mu_q_g_D2)
hamiltonian_D2.add_mu_q_block('e', mu_q_e_D2)



#ham = pylcp.hamiltonian(Hg, He, mugq, mueq, dijq, mass=mass, gamma=gamma, k=k)
det_1 = np.linspace(0,-20,100)
vc = x0/(t0*100)
# Define the atomic Hamiltonian:
H_g_D2, mu_q_g_D2 = pylcp.hamiltonians.hyperfine_coupled(
0, 5/2, 0,  	-0.2592,
    Ahfs = 0, Bhfs=0, Chfs=0,
    muB=1)# ground state 1s0
H_e_D2, mu_q_e_D2 = pylcp.hamiltonians.hyperfine_coupled(
1, 5/2, 1.035,	-0.2592,
    Ahfs=59.52*1e6*t0,Bhfs = 601.87*1e6*t0 , Chfs= 0,
    muB=1) #excited state 1p1

dijq_D2 = pylcp.hamiltonians.dqij_two_hyperfine_manifolds(0, 1, 5/2)

E_e_D2 = np.unique(np.diagonal(H_e_D2))
E_g_D2 = np.unique(np.diagonal(H_g_D2))
hamiltonian_D2 = pylcp.hamiltonian(mass = mass)
hamiltonian_D2.add_H_0_block('g', H_g_D2)
hamiltonian_D2.add_H_0_block('e', H_e_D2)
hamiltonian_D2.add_d_q_block('g', 'e', dijq_D2, gamma = gamma, k = k)
hamiltonian_D2.add_mu_q_block('g', mu_q_g_D2)
hamiltonian_D2.add_mu_q_block('e', mu_q_e_D2)



#ham = pylcp.hamiltonian(Hg, He, mugq, mueq, dijq, mass=mass, gamma=gamma, k=k)

bf = lambda R: cts.value('Bohr magneton')*b_tot.getB(R*10*x0)*t0/cts.hbar
#l = 16/x0
#l = 30/x0
a
bf2 = lambda R: mag(R,b_0,alpha,16/x0)
linGrad = pylcp.magField(lambda R: bf(R))

In [ ]:
y = np.linspace(0,2*30,100)/x0
magg= []
magg2 = []
for i in y:
    magg.append(bf(np.array([0,0,i]))[0])
    magg2.append(bf2(np.array([0,0,i]))[0])
plt.plot(y*x0,np.array(magg)*1e3)
plt.plot(y*x0,np.array(magg2)*1e3)

In [ ]:
32/0.05*400

# calculating and ploting force proifle

In [ ]:
#when using ideal magfield it has to be off due
det = np.linspace(-30,0,100)

force_profiles = {}
det = np.linspace(0,30,50)
j  = 0 
b = 0

laserBeams = pylcp.laserBeams([
{'kvec':np.array([0, 0, -1]), 's': 5,'pol_coord':'spherical', 'pol':np.array([1,0,1]), 'delta': (E_e_D2[2]-E_g_D2[0]) - det[24]*gamma},],
beam_type=pylcp.infinitePlaneWaveBeam) 
rateeq = pylcp.rateeq(laserBeams, linGrad, hamiltonian_D2, include_mag_forces=False)
x = np.arange(0, 2*L, 0.05)/(x0)
v = np.arange(-100, 410,1)/vc


X, V = np.meshgrid(x, v)

Rvec = np.array([np.zeros(X.shape), np.zeros(X.shape), X])
Vvec = np.array([np.zeros(V.shape), np.zeros(V.shape), V])

rateeq.generate_force_profile(Rvec, Vvec, name='Fz', progress_bar=True)

force_profiles[f'{j}'] = rateeq.profile['Fz'].F[2]

b = b +1
print(b)

In [ ]:
5*1e-3*100

In [ ]:

fig,ax = plt.subplots(1, 1, figsize=(8, 4)) 
cmap_reversed = plt.cm.get_cmap('viridis_r')
Fc = gammalab*cts.hbar*klab*100
im1 = ax.contourf(np.array(x)*x0, np.array(v)*vc, rateeq.profile['Fz'].F[2]*Fc,vmin = -1.1e-19,vmax = 0)
fig.subplots_adjust(left=0.08, wspace=0.2)
range_l = np.linspace(-1.1e-19,0,10)
cb1 = plt.colorbar(im1,ticks = range_l)
cb1.set_label('$F(N)$')
ax.set_xlabel('$x$ (cm)')
ax.set_ylabel('$v (ms^{-1})$')
plt.ylim([0,400])
plt.title('Contour plot of position vs velocity and force')

In [ ]:
(150*1.4e6)/(klab*100)

In [ ]:
# check of ithe inportlation of force profile
# Create the interpolator

f_interp = RegularGridInterpolator((np.array(v)*vc,np.array(x)*x0/100),rateeq.profile['Fz'].F[2]*Fc, method='linear')

L = 16
# Generate a fine grid for plotting
x_fine = np.arange(0, 60, 0.05)/100# Fine grid for x-axis

v_fine =  np.arange(0,400 ,0.5) # Fine grid for v-axis
V,X  = np.meshgrid(v_fine, x_fine)  # Create a 2D grid

# Interpolate on the fine grid
points = np.array([V.flatten(), X.flatten()]).T  # Flatten grid into (n, 2) points
Z = f_interp(points).reshape(X.shape)  # Interpolated values reshaped to grid

# Create the contour plot
plt.figure(figsize=(8, 6))
contour = plt.contourf(X*100, V, Z, levels=20, cmap='viridis')

plt.colorbar(contour, label="Interpolated Values")
plt.xlabel("X-axis (x)")
plt.ylabel("Y-axis (v)")
plt.title("Contour Plot of Interpolated Data")
plt.show()

# Modeling velocity distribution of Yb-173 atoms out of 450 degree oven

In [ ]:
def find_nearest_value(lst, target):
    nearest = min(lst, key=lambda x: abs(x - target))
    return nearest

In [ ]:
# Constants
k_B = cts.k  # Boltzmann constant in J/K
 # Atomic mass of Yb-173
mass_kg = mass_lab  # Convert to kilograms
temperature = 405 + 273.15  # Convert Celsius to Kelvin

# Velocity Probability Function
def p(m, T, v):
    return (m / (k_B * T))**2 * np.exp(-m * v**2 / (2 * k_B * T)) * (v**3) / 2

In [ ]:
class MaxwellBoltzmann(rv_continuous):
    def _pdf(self, v):
        return p(mass_kg, temperature, v)

# Define the custom distribution
maxwell_boltzmann = MaxwellBoltzmann(a=0, name="Maxwell-Boltzmann")

In [ ]:
size = 10000# Number of samples
velocities = maxwell_boltzmann.rvs(size=size)
velocities = [i for i in velocities if i < 400]
# Display the sampled velocities
print(len(velocities))
plt.hist(velocities, bins=100, density=True)
plt.grid()

# calculating trajectories 

In [ ]:
f = rateeq.profile['Fz'].F[2]*Fc
f_interp = RegularGridInterpolator((np.array(v)*vc,np.array(x)),f,method = 'linear')

def simulate_trajectory(v_0, t_span,f_interp, mass_lab):
    x_t, v_t, t = [], [], []
    s, di = 0, 0  # Initial conditions

    for q in range(len(t_span)):
        dt = t_span[q] - di

        if s > 0.3 or v_0 < 0 or v_0 >= 400:  # Break condition
            break

        # Interpolate the force profile
        
        a = f_interp((v_0, s)) / mass_lab


        # Update position and velocity

        s = (1/2)*a*(dt**2) + v_0*dt + s
        
        v_0 = v_0 + a*dt

        # Append values
        x_t.append(s)
        v_t.append(v_0)
        t.append(t_span[q])
        di = t_span[q]
        #print(v_t)
       

    return v_t, x_t, t

In [ ]:
t_max = 0.05 
t_len = [100,1000,10000,100000]
v_t_z = []
z_t_z = []
t_lenn = []
for i in t_len:
    t_span = np.linspace(0, t_max, i)
    v_t,z_t,t = simulate_trajectory(100, t_span,f_interp, mass_lab)
    v_t_z.append(v_t)
    t_lenn.append(t)
    z_t_z.append(z_t)
fig,ax = plt.subplots(1,4,figsize=(15,5))
ax[0].plot(np.array(t_lenn[0])*1000,np.array(v_t_z[0]))
ax[1].plot(np.array(t_lenn[1])*1000,np.array(v_t_z[1]))
ax[2].plot(np.array(t_lenn[2])*1000,np.array(v_t_z[2]))
ax[3].plot(np.array(t_lenn[3])*1000,np.array(v_t_z[3]))
ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('velocity (cm)')
ax[1].set_xlabel('Time (ms)')
ax[1].set_ylabel('velocity (cm)')
ax[2].set_xlabel('Time (ms)')
ax[2].set_ylabel('velocity (cm)')
ax[0].set_title('Velocity vs Time for 100 steps')
ax[1].set_title('Velocity vs Time for 1000 steps')
ax[2].set_title('Velocity vs Time for 10000 steps')
ax[3].set_title('Velocity vs Time for 100000 steps')
plt.tight_layout()


In [ ]:
v_z = []
z_t_n = []
capture_effeciecency = []
v_z_n = []
t_n_all = []
t_max = 0.05
t_span = np.linspace(0, t_max, 1000)

#for key in force_profiles:
f_interp = RegularGridInterpolator((np.array(v)*vc,np.array(x)*x0/100),rateeq.profile['Fz'].F[2]*Fc,method='linear')
v_z = []
z_t = []
t_n = []
for i in range(len(velocities)):
        v_t,x_t,t = simulate_trajectory(velocities[i], t_span, f_interp,mass_lab)
        v_z.append(v_t)
        z_t.append(x_t)

c1 = 0
for i in v_z:
        if i[-1] < 144:
                c1 = c1+1 
print(c1)
capture_effeciecency.append(c1)
#plt.plot(det,capture_effeciecency)

In [ ]:
final_velocity = []
initial_velocity = []
for i in v_z:
    final_velocity.append(i[-1])
    initial_velocity.append(i[0])

In [ ]:
c = 0
for i in initial_velocity:
    if i < 144:
        c = c+1
print(c*100/len(velocities))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 8))

ax[0].hist(final_velocity, bins=100, density=True)
ax[1].hist(initial_velocity, bins=100, density=True)

ax[0].grid()
ax[1].grid()
ax[0].set_title('Vel Distribution after zeeman slower(idealB,30 cm),len(t_span) = 1000000')
ax[1].set_title('Vel Distribution before zeeman slower(idealB,30 cm)')
ax[0].set_ylabel('Frequency of velocity ')
ax[1].set_xlabel('Velocity (m/s)' )
ax[0].set_xlabel('Velocity (m/s)')
ax[1].set_ylabel('Frequency of velocity')
ax[0].set_ylim([0,0.0035])

ax[1].set_ylim([0,0.0035])
plt.tight_layout()


In [ ]:
bump_s = [final_velocity.index(i) for i in final_velocity if i < 200 and i > 100]
bump_b = [final_velocity.index(i) for i in final_velocity if i > 375 and i < 380]

In [ ]:
bump_ts = []
t_ss = []
bump_zs = []
for j in bump_s:
    bump_ts.append(v_z[j])
    bump_zs.append(z_t[j])
    t_ss.append(t_span[:len(v_z[j])])


In [ ]:
bump_tb = []
t_sb = []
bump_zb = []
for j in bump_b:
    bump_tb.append(v_z[j])
    bump_zb.append(z_t[j])
    t_sb.append(t_span[:len(v_z[j])])

In [ ]:
len(bump_tb)

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(10,10))
for  i in range(len(bump_ts)):
    ax[0][1].plot(t_ss[i]*1e3,bump_ts[i])
    ax[0][0].plot(t_ss[i]*1e3,np.array(bump_zs[i])*100)
for i in range(len(bump_tb)):
    ax[1][1].plot(t_sb[i]*1e3,bump_tb[i])
    ax[1][0].plot(t_sb[i]*1e3,np.array(bump_zb[i])*100)
ax[0][0].set_xlabel('Time (ms)')
ax[0][0].set_ylabel('Position (cm)')
ax[0][1].set_xlabel('Time (ms)')
ax[0][1].set_ylabel('Velocity (m/s)')
ax[0][0].set_title('Position vs Time(first bump)')
ax[0][1].set_title('Velocity vs Time(first bump)')
ax[1][0].set_xlabel('Time (ms)')
ax[1][0].set_ylabel('Position (cm)')
ax[1][1].set_xlabel('Time (ms)')
ax[1][1].set_ylabel('Velocity (m/s)')
ax[1][0].set_title('Position vs Time(Second Bump)')
ax[1][1].set_title('Velocity vs Time(Second Bump)')
plt.tight_layout()

In [ ]:
other = [initial_velocity.index(i) for i in  initial_velocity if  i > 350 and i < 400 ]
other_t = []
t_so = []
other_z = []
for j in other:
    other_t.append(v_z[j])
    other_z.append(z_t[j])
    t_so.append(t_span[:len(v_z[j])])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(8,5))
for  i in range(len(other_t)):
    ax[0].plot(t_so[i]*1e3,other_t[i])
    ax[1].plot(t_so[i]*1e3,np.array(other_z[i])*100)
ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('Velocity (m/s)')
ax[1].set_xlabel('Time (ms)')
ax[1].set_ylabel('Position (cm)')
ax[0].set_title('Velocity vs Time(other)')
ax[1].set_title('Position vs Time(other)')


In [ ]:
for i in range(len(other_t)):
    plt.plot(other_z[i],other_t[i])

plt.xlabel('Position (cm)')
plt.ylabel('Velocity (m/s)')
plt.title('Velocity vs Position(V_f all velocities)')

In [ ]:
z = np.linspace(0,400)
y = z
plt.scatter(initial_velocity,final_velocity,s = 0.008)
plt.plot(z,y,'r')
plt.xlabel('Initial Velocity (m/s)')
plt.ylabel('Final Velocity (m/s)')
plt.grid()

In [ ]:
x = np.linspace(0,400)
y = x
plt.scatter(initial_velocity,final_velocity,s = 0.008)
plt.plot(x,y,'r')
plt.xlabel('Initial Velocity (m/s)')
plt.ylabel('Final Velocity (m/s)')
plt.xlim([375,395])
plt.ylim([340,400])
plt.grid()

In [ ]:
for i in range(len(bump_t)):
    plt.plot(np.array(bump_z[i])*100,bump_t[i])
    plt.xlabel('Position (cm)')
    plt.ylabel('Velocity (m/s)')
    plt.title('Velocity vs Position of the anomalous bump')